# Knapsack Optimization using Hybrid GA + Local Search

### Create the dataset
OR-Library (via Florida State University mirror)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

def scrape_knapsack_problem_instances():
    base_url = "https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/"

    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Get all problem prefixes (p01, p02, etc.)
    problem_prefixes = set()
    for link in soup.find_all('a', href=True):
        href = link['href']
        match = re.match(r'(p\d+)', href)
        if match:
            problem_prefixes.add(match.group(1))

    problem_instances = []

    for prefix in sorted(problem_prefixes):
        print(f"Processing {prefix}...")

        try:
            # Initialize problem data for this prefix
            problem_data = {'problem_id': prefix}

            # Download all file types for this problem
            file_types = ['c', 'w', 'p', 's']

            for file_type in file_types:
                file_url = f"{base_url}{prefix}_{file_type}.txt"
                file_response = requests.get(file_url)

                if file_response.status_code == 200:
                    content = file_response.text.strip()

                    if file_type == 'c':  # Capacity
                        problem_data['capacity'] = int(content)
                    else:
                        numbers = []
                        for line in content.split('\n'):
                            line = line.strip()
                            if line and not line.startswith('#'):
                                try:
                                    numbers.extend([int(x) for x in line.split()])
                                except ValueError:
                                    continue

                        if file_type == 'w':
                            problem_data['weights'] = numbers
                        elif file_type == 'p':
                            problem_data['values'] = numbers
                        elif file_type == 's':
                            problem_data['optimal_solution'] = numbers

            # Only add if we have the essential data (weights and profits)
            if 'weights' in problem_data and 'values' in problem_data:
                problem_instances.append(problem_data)
            else:
                print(f"Missing essential data for {prefix}")

        except Exception as e:
            print(f"Error processing {prefix}: {e}")
            continue

    return problem_instances

# Run the scraper
print("Scraping knapsack problem instances...")
problem_instances = scrape_knapsack_problem_instances()

# Create the DataFrame
df = pd.DataFrame(problem_instances)

# Reorder columns for better readability
column_order = ['problem_id', 'capacity', 'weights', 'values']
if 'optimal_solution' in df.columns:
    column_order.append('optimal_solution')
df = df[column_order]

print(f"\nCreated DataFrame with {len(df)} unique problem instances")
print("Problem IDs:", df['problem_id'].tolist())

# Save to CSV
df.to_csv('knapsack_problem_instances.csv', index=False)
print("\nSaved to 'knapsack_problem_instances.csv'")

# Display all problems briefly
print("\nAll problem instances:")
for _, row in df.iterrows():
    print(f"{row['problem_id']}: {len(row['weights'])} items, capacity {row['capacity']}")

Scraping knapsack problem instances...
Processing p01...
Processing p02...
Processing p03...
Processing p04...
Processing p05...
Processing p06...
Processing p07...
Processing p08...

Created DataFrame with 8 unique problem instances
Problem IDs: ['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p07', 'p08']

Saved to 'knapsack_problem_instances.csv'

All problem instances:
p01: 10 items, capacity 165
p02: 5 items, capacity 26
p03: 6 items, capacity 190
p04: 7 items, capacity 50
p05: 8 items, capacity 104
p06: 7 items, capacity 170
p07: 15 items, capacity 750
p08: 24 items, capacity 6404180


In [2]:
import ast

df = pd.read_csv("knapsack_problem_instances.csv")
df['weights'] = df['weights'].apply(ast.literal_eval)
df['values'] = df['values'].apply(ast.literal_eval)
if 'optimal_solution' in df.columns:
    df['optimal_solution'] = df['optimal_solution'].apply(ast.literal_eval)

df.head(100)

,problem_id,capacity,weights,values,optimal_solution
0,p01,165,"[23, 31, 29, 44, 53, 38, 63, 85, 89, 82]","[92, 57, 49, 68, 60, 43, 67, 84, 87, 72]","[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]"
1,p02,26,"[12, 7, 11, 8, 9]","[24, 13, 23, 15, 16]","[0, 1, 1, 1, 0]"
2,p03,190,"[56, 59, 80, 64, 75, 17]","[50, 50, 64, 46, 50, 5]","[1, 1, 0, 0, 1, 0]"
3,p04,50,"[31, 10, 20, 19, 4, 3, 6]","[70, 20, 39, 37, 7, 5, 10]","[1, 0, 0, 1, 0, 0, 0]"
4,p05,104,"[25, 35, 45, 5, 25, 3, 2, 2]","[350, 400, 450, 20, 70, 8, 5, 5]","[1, 0, 1, 1, 1, 0, 1, 1]"
5,p06,170,"[41, 50, 49, 59, 55, 57, 60]","[442, 525, 511, 593, 546, 564, 617]","[0, 1, 0, 1, 0, 0, 1]"
6,p07,750,"[70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110,...","[135, 139, 149, 150, 156, 163, 173, 184, 192, ...","[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]"
7,p08,6404180,"[382745, 799601, 909247, 729069, 467902, 44328...","[825594, 1677009, 1676628, 1523970, 943972, 97...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ..."


### Declare the initial population

In [3]:
import random , os
os.environ['PYTHONHASHSEED'] = '42'
np.random.seed(42)
random.seed(42)

In [5]:
bags = len(df)
pop_size = 100
num_parents = 50
mutation_rate = 0.1
mutation_proba = 0.01
elitism_rate = 0.2

In [6]:
def initialize_population(bag, pop_size):
    """Create initial population for GA"""
    n_items = len(bag['weights'])
    population = np.random.randint(0, 2, (pop_size, n_items))
    return population

### Fitness Score

In [7]:
# Calculate the fitness score
def call_fitness(weight, value, population, threshold):
    fitness = np.empty(population.shape[0])
    for i in range(population.shape[0]):
        S1 = np.sum(population[i] * value)
        S2 = np.sum(population[i] * weight)
        if S2 <= threshold:
            fitness[i] = S1
        else :
            fitness[i] = 0
    return fitness.astype(int)

### Parents selection

1- séléction par tournoi

In [8]:
# tournament tournamant_selection
def tournamant_selection(fitness, population, num_parents, k=10):
    parents = []
    for _ in range(num_parents):
        selected_idx = np.random.choice(range(len(population)), k, replace=False)
        best_idx = selected_idx[np.argmax(fitness[selected_idx])]
        parents.append(population[best_idx])
    return np.array(parents)

2-sélection à la roulette

In [9]:
def selection_roulette(fitness, population, num_parents):
    parents = []
    fitness = np.array(fitness, dtype=float)
    total_fitness = np.sum(fitness)

    # Si toutes les fitness sont nulles, donner des chances égales
    if total_fitness == 0:
        probs = np.ones(len(fitness)) / len(fitness)
    else:
        probs = fitness / total_fitness

    for _ in range(num_parents):
        idx = np.random.choice(len(population), p=probs)
        parents.append(population[idx])

    return np.array(parents)

3-selection par rang

In [10]:
def selection_rank(fitness, population, num_parents):
    parents = []
    fitness = np.array(fitness, dtype=float)

    # Trier les individus par fitness
    ranks = np.argsort(fitness)
    # Attribuer une probabilité proportionnelle au rang (1 = faible, N = meilleur)
    rank_values = np.arange(1, len(fitness) + 1)
    probs = rank_values / np.sum(rank_values)

    for _ in range(num_parents):
        idx = np.random.choice(len(population), p=probs)
        parents.append(population[ranks[idx]])

    return np.array(parents)

### Crossover

1- croisement multiples

In [11]:
def crossover(parents):
    offsprings = []
    def three_point_crossover(parent1, parent2):
        if len(parent1) == len(parent2) :
            # Generate three unique crossover points
            points = sorted(random.sample(range(1, len(parent1)), 3))
            offspring1 = np.concatenate((parent1[:points[0]],parent2[points[0]:points[1]],parent1[points[1]:points[2]],parent2[points[2]:]))
            offspring2 = np.concatenate((parent2[:points[0]],parent1[points[0]:points[1]],parent2[points[1]:points[2]],parent1[points[2]:]))
            return offspring1, offspring2

    def four_point_crossover(parent1, parent2):
        if len(parent1) == len(parent2) :
            # Generate three unique crossover points
            points = sorted(random.sample(range(1, len(parent1)), 4))
            offspring1 = np.concatenate((parent1[:points[0]],parent2[points[0]:points[1]],parent1[points[1]:points[2]],parent2[points[2]:points[3]],parent1[points[3]:]))
            offspring2 = np.concatenate((parent2[:points[0]],parent1[points[0]:points[1]],parent2[points[1]:points[2]],parent1[points[2]:points[3]],parent2[points[3]:]))
            return offspring1, offspring2

    for i in range(0, len(parents) - 1, 2):
        if len(parents[i]) <= 12 :
            offspring1,offspring2 = three_point_crossover(parents[i], parents[i+1])
        else :
            offspring1,offspring2 = four_point_crossover(parents[i], parents[i+1])
        offsprings.extend([offspring1, offspring2])

    return offsprings

2-Croisement simple avec un seul point de coupure

In [12]:
def one_point_crossover(parents):
    offsprings = []
    for i in range(0, len(parents) - 1, 2):
        parent1, parent2 = parents[i], parents[i+1]
        if len(parent1) == len(parent2):
            point = random.randint(1, len(parent1)-1)
            offspring1 = np.concatenate((parent1[:point], parent2[point:]))
            offspring2 = np.concatenate((parent2[:point], parent1[point:]))
            offsprings.extend([offspring1, offspring2])
    return np.array(offsprings)

3-uniform crossover

In [13]:
def uniform_crossover(parents, p=0.5):
    offsprings = []
    for i in range(0, len(parents) - 1, 2):
        parent1, parent2 = parents[i], parents[i+1]
        offspring1, offspring2 = parent1.copy(), parent2.copy()
        for j in range(len(parent1)):
            if random.random() < p:
                offspring1[j], offspring2[j] = offspring2[j], offspring1[j]
        offsprings.extend([offspring1, offspring2])
    return np.array(offsprings)

### Mutation

1- bit flip mutation

In [14]:
def bit_flip_mutation(offspring, mutation_rate, mutation_proba):
    mutated_offsprings = []
    num_offspring = int(len(offspring) * mutation_rate )
    selected_indices = np.random.choice(len(offspring), size=num_offspring, replace=False)
    for idx in selected_indices:
        offsp = offspring[idx].copy()
        for i in range(len(offsp)):
            if random.uniform(0, 1) < mutation_proba:
                offsp[i] = 1 - offsp[i]
        mutated_offsprings.append(offsp)
    return np.array(mutated_offsprings)

2-swap mutation

In [26]:
def swap_mutation(offspring, mutation_rate, mutation_proba=None):
    mutated_offsprings = []
    num_offspring = int(len(offspring) * mutation_rate)
    selected_indices = np.random.choice(len(offspring), size=num_offspring, replace=False)

    for idx in selected_indices:
        offsp = offspring[idx].copy()
        i, j = np.random.choice(len(offsp), 2, replace=False)
        offsp[i], offsp[j] = offsp[j], offsp[i]
        mutated_offsprings.append(offsp)

    return np.array(mutated_offsprings)

### Replacement

In [16]:
def elitist_replacement(fitness, population, elitism_rate, offspring, mutated_offsprings):
    best_pop_size = int(elitism_rate * len(population))
    # Get indices of individuals sorted by fitness descending
    elite_indices = np.argsort(fitness)[-best_pop_size:]
    best_pop = [population[i] for i in elite_indices]

    # Combine elites with offspring and mutated offspring
    new_population = list(offspring) + list(mutated_offsprings) + best_pop

    # Optionally truncate or sample if new_population size exceeds original
    new_population = new_population[:len(population)]
    new_population = np.array(new_population[:len(population)])
    return new_population

### Fitness evolution plot

In [17]:
import matplotlib.pyplot as plt
def plot_fitness(fitness_history):
    fitness_history_mean = [np.mean(fitness) for fitness in fitness_history]
    fitness_history_max = [np.max(fitness) for fitness in fitness_history]
    plt.plot(list(range(num_generations)), fitness_history_mean, label = 'Mean Fitness')
    plt.plot(list(range(num_generations)), fitness_history_max, label = 'Max Fitness')
    plt.legend()
    plt.title('Fitness through the generations')
    plt.xlabel('Generations')
    plt.ylabel('Fitness')
    plt.show()

## Procédé de l'algorithme génétique ##

In [27]:
selection_methods = {
    "tournament": tournamant_selection,
    "roulette": selection_roulette,
    "rank": selection_rank
}

crossover_methods = {
    "multi_point": crossover,
    "one_point": one_point_crossover,
    "uniform": uniform_crossover
}

mutation_methods = {
    "bit_flip": bit_flip_mutation,
    "swap": swap_mutation
}

In [28]:
num_generations = 50
pop_size = 100
num_parents = 50
mutation_rate = 0.1
mutation_proba = 0.01
elitism_rate = 0.2

def ga_accuracy(true_sol, predicted_sol):
    accuracies = []
    for true, pred in zip(true_sol, predicted_sol):
        true = np.array(true)
        pred = np.array(pred)
        min_len = min(len(true), len(pred))
        acc = np.sum(true[:min_len] == pred[:min_len]) / min_len
        accuracies.append(acc)

    return np.mean(accuracies)

results = []

for sel_name, sel_func in selection_methods.items():
    for cross_name, cross_func in crossover_methods.items():
        for mut_name, mut_func in mutation_methods.items():
            predicted_sol = []
            true_sol_list = []

            # Boucle sur tous les sacs
            for i in range(len(df)):
                bag = df.iloc[i]
                weights = bag['weights']
                values = bag['values']
                capacity = bag['capacity']
                true_sol_list.append(bag['optimal_solution'])

                population = initialize_population(bag, pop_size)
                gen = 0

                while gen < num_generations:
                    fitness_scores = call_fitness(weights, values, population, capacity)
                    parents = sel_func(fitness_scores, population, num_parents)
                    offspring = cross_func(parents)
                    mutated_offspring = mut_func(offspring, mutation_rate, mutation_proba)
                    population = elitist_replacement(fitness_scores, population, elitism_rate, offspring, mutated_offspring)
                    gen += 1

                # Sauvegarder la meilleure solution
                fitness_last_gen = call_fitness(weights, values, population, capacity)
                best_idx = np.argmax(fitness_last_gen)
                predicted_sol.append(population[best_idx])

            # Calculer l’accuracy
            acc = ga_accuracy(true_sol_list, predicted_sol)
            results.append({
                "Selection": sel_name,
                "Crossover": cross_name,
                "Mutation": mut_name,
                "Accuracy": acc
            })

In [29]:
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="Accuracy", ascending=False))

     Selection    Crossover  Mutation  Accuracy
9     roulette    one_point      swap  0.918750
2   tournament    one_point  bit_flip  0.918750
16        rank      uniform  bit_flip  0.914583
12        rank  multi_point  bit_flip  0.908333
11    roulette      uniform      swap  0.903125
6     roulette  multi_point  bit_flip  0.887500
17        rank      uniform      swap  0.886458
8     roulette    one_point  bit_flip  0.865625
1   tournament  multi_point      swap  0.853571
4   tournament      uniform  bit_flip  0.851042
0   tournament  multi_point  bit_flip  0.835417
15        rank    one_point      swap  0.833780
14        rank    one_point  bit_flip  0.831696
13        rank  multi_point      swap  0.825446
5   tournament      uniform      swap  0.811905
7     roulette  multi_point      swap  0.800446
10    roulette      uniform  bit_flip  0.795238
3   tournament    one_point      swap  0.724405
